[Reference](https://medium.com/@labdheesheth/visualizing-geographical-data-using-geoplotlib-d732953abcd5)

In [1]:
pip install geoplotlib

  Created wheel for geoplotlib: filename=geoplotlib-0.3.2-py3-none-any.whl size=24572 sha256=beaab1fd91c8f989d893fb867017df52db5cd0c86252f4819a790c7ba3139fb2
  Stored in directory: /root/.cache/pip/wheels/f0/b7/30/9dca97573fcf2c600c00f8cb1cd8036e0a3214b92d3639d6db
Successfully built geoplotlib


In [6]:
pip install pyvirtualdisplay

In [11]:
!apt-get install -y xvfb x11-utils

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libxxf86dga1
Suggested packages:
  mesa-utils
The following NEW packages will be installed:
  libxxf86dga1 x11-utils xvfb
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 994 kB of archives.
After this operation, 2,981 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 x11-utils amd64 7.7+3build1 [196 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 994 kB in 1s (1,371 kB/s)
Selecting previously unselected package libxxf86dga1:amd64.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../libxxf86dga1_2%3a1.1.4-1_amd64.deb ...
Unpacking libxxf86dga1:amd64 (2:

In [7]:
from pyvirtualdisplay import Display

In [12]:
display = Display(visible=False, size=(400, 300))
display.start()

# 1. Dot Map:

In [2]:
import geoplotlib
from geoplotlib.utils import read_csv
import pandas as pd

data = read_csv('https://raw.githubusercontent.com/andrea-cuttone/geoplotlib/master/examples/data/bus.csv')
geoplotlib.dot(data)
geoplotlib.show()

# 2. 2D histograms:

In [3]:
import geoplotlib
from geoplotlib.utils import read_csv, BoundingBox

data = read_csv('https://raw.githubusercontent.com/andrea-cuttone/geoplotlib/master/examples/data/opencellid_dk.csv')
geoplotlib.hist(data, colorscale='sqrt', binsize=8)
geoplotlib.set_bbox(BoundingBox.DK)
geoplotlib.show()

# 3. Heatmap:

In [14]:
import geoplotlib
from geoplotlib.utils import read_csv, BoundingBox, DataAccessObject
import pandas as pd

df = pd.read_csv('district wise population and centroids.csv')
df.columns = ['State', 'District', 'lat', 'lon', 'Population in 2001',
       'Population in 2011']
geoplotlib.kde(df, bw=7, cut_below=1e-4)

geoplotlib.set_bbox(BoundingBox.KBH)
geoplotlib.show()

# 4. Spatial graph:

In [15]:
import geoplotlib
from geoplotlib.utils import read_csv

data = read_csv('https://raw.githubusercontent.com/andrea-cuttone/geoplotlib/master/examples/data/flights.csv')
geoplotlib.graph(data,
                 src_lat='lat_departure',
                 src_lon='lon_departure',
                 dest_lat='lat_arrival',
                 dest_lon='lon_arrival',
                 color='hot_r',
                 alpha=16,
                 linewidth=2)
geoplotlib.show()

# 5. Moving points:

In [16]:
from geoplotlib.layers import BaseLayer
from geoplotlib.core import BatchPainter
import geoplotlib
from geoplotlib.colors import colorbrewer
from geoplotlib.utils import epoch_to_str, BoundingBox, read_csv


class TrailsLayer(BaseLayer):

    def __init__(self):
        self.data = read_csv('https://github.com/andrea-cuttone/geoplotlib/blob/master/examples/data/taxi.csv')
        self.cmap = colorbrewer(self.data['taxi_id'], alpha=220)
        self.t = self.data['timestamp'].min()
        self.painter = BatchPainter()


    def draw(self, proj, mouse_x, mouse_y, ui_manager):
        self.painter = BatchPainter()
        df = self.data.where((self.data['timestamp'] > self.t) & (self.data['timestamp'] <= self.t + 15*60))

        for taxi_id in set(df['taxi_id']):
            grp = df.where(df['taxi_id'] == taxi_id)
            self.painter.set_color(self.cmap[taxi_id])
            x, y = proj.lonlat_to_screen(grp['lon'], grp['lat'])
            self.painter.points(x, y, 10)

        self.t += 2*60

        if self.t > self.data['timestamp'].max():
            self.t = self.data['timestamp'].min()

        self.painter.batch_draw()
        ui_manager.info(epoch_to_str(self.t))


    def bbox(self):
        return BoundingBox(north=40.110222, west=115.924463, south=39.705711, east=116.803369)


geoplotlib.add_layer(TrailsLayer())
geoplotlib.show()